# Cough Detection Feature Extraction

This notebook serves as a foundation on what and why each features are extracted from multimodal biosignals (audio + IMU sensors) for machine learning-based cough detection.

## Configuration

- **Window size**: 0.4 seconds (6400 audio samples @ 16kHz, 40 IMU samples @ 100Hz)
- **Data augmentation**: Random temporal shifts (aug_factor=2) to increase dataset diversity

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats, signal
import librosa
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

if os.path.exists("/kaggle/usr/lib/"):
    # Load from Kaggle as utility scripts
    from edge_ai_cough_count_helpers import * # pyright: ignore[reportMissingImports]
    from edge_ai_cough_count_dataset_gen import * # pyright: ignore[reportMissingImports]
    from edge_ai_cough_count_features import extract_audio_features, extract_imu_features # pyright: ignore[reportMissingImports]
else:
    # Add src directory to path
    sys.path.append(os.path.abspath('../src'))
    from helpers import *
    from dataset_gen import *
    from features import extract_audio_features, extract_imu_features

print("✓ All imports successful")

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Constants from paper
WINDOW_LEN = 0.4  # 0.4 second windows
AUG_FACTOR = 2    # Data augmentation factor

## Why Feature Extraction?

You might wonder: **Why not just feed the raw audio and IMU signals directly into a machine learning model?**

Here's why feature extraction is essential:

1. **Size reduction**: A 0.4-second window contains 6,400 audio samples. Instead of processing all 6,400 values, we extract just 65 meaningful features that capture the "essence" of the sound.

2. **Noise reduction**: Raw signals contain a lot of irrelevant information and noise. Features focus on what matters - patterns that distinguish coughs from other sounds.

3. **Better learning**: Machine learning models learn faster and more accurately from compact, meaningful features than from thousands of raw signal values.

**Analogy**: Imagine describing a person. Instead of showing a full high-resolution photo (raw signal), you describe key features: height, eye color, age, build (extracted features). Both convey information, but features are more compact and focused on what matters.

In this notebook, we'll extract **105 features** that capture the unique characteristics of cough sounds and motions.

## Feature Categories Overview

Our 105 features are organized into two main groups:

| Category | Count | What It Captures | Example Use Case |
|----------|-------|------------------|------------------|
| **Audio Features** | **65** | Sound characteristics from microphone | |
| → MFCC | 52 | Sound "texture" or "timbre" | Distinguishing cough from speech |
| → Spectral | 10 | Frequency distribution | High-pitched vs low-pitched sounds |
| → Time-domain | 3 | Loudness and sharpness | Sudden bursts vs gradual sounds |
| **IMU Features** | **40** | Body motion from sensors | |
| → Accelerometer | 20 | Linear motion (chest/body movement) | Detecting physical cough motion |
| → Gyroscope | 20 | Rotational motion (head/neck) | Detecting head movement during cough |
| **Total** | **105** | Complete multimodal signature | Robust cough detection |

### Why Use Both Audio AND Motion Sensors?

**Multimodal sensing is more robust than either modality alone:**

- **When audio struggles**: Noisy environments (music, traffic, other people talking)
  - IMU still captures the physical motion of coughing
  
- **When IMU struggles**: Sitting still, gentle cough, other body movements
  - Audio captures the distinctive cough sound
  
- **Together**: The combination is much more accurate than either sensor alone

In the walkthrough below, you'll see exactly how each feature category works and why it helps detect coughs!

## Understanding Sampling and Sample Rate

Before we can extract features from audio and IMU signals, we need to understand how these **continuous real-world signals** become **discrete digital data** that computers can process.

### What is Sampling?

**Sampling** is the process of converting a continuous analog signal (like sound waves or motion) into a discrete digital signal by taking measurements at regular intervals.

**Analogy**: Imagine you're watching a car drive by:
- **Continuous (analog)**: The car moves smoothly through space
- **Discrete (digital)**: You take snapshots every 0.1 seconds to track its position

The snapshots are your **samples**, and how often you take them is your **sampling rate**.

### What is Sample Rate (Sampling Frequency)?

**Sample rate** (also called **sampling frequency**) is the number of samples taken per second, measured in **Hertz (Hz)**.

**Examples:**
- **16,000 Hz** means 16,000 samples per second
- **100 Hz** means 100 samples per second

**Higher sample rate** = more snapshots per second = more detailed representation of the signal

### Our Sensors and Their Sample Rates

In this cough detection project, we use two types of sensors with different sample rates:

| Sensor | Sample Rate | Samples per Second | What It Captures |
|--------|-------------|-------------------|------------------|
| **Microphone** | 16,000 Hz | 16,000 | Sound waves (cough, speech, ambient noise) |
| **IMU (Accelerometer + Gyroscope)** | 100 Hz | 100 | Body motion (chest movement, head rotation) |

**Why different rates?**
- **Audio** changes very rapidly (sound frequencies range from 20 Hz to 20,000 Hz), so we need a high sample rate to capture it accurately
- **Body motion** changes much more slowly than sound waves, so 100 samples/second is sufficient

### From Real World to Digital Samples

Here's what happens in practice:

**When you cough:**

1. **Microphone** measures air pressure 16,000 times per second → Creates an array of 16,000 numbers per second
2. **Accelerometer** measures chest acceleration 100 times per second → Creates an array of 100 numbers per second
3. **Gyroscope** measures body rotation 100 times per second → Creates an array of 100 numbers per second

**A 1-second recording produces:**
- Audio: 16,000 values (numbers representing sound amplitude)
- IMU: 100 values for each of the 6 channels (3 accelerometer axes + 3 gyroscope axes) = 600 values total

These arrays of numbers are what we'll work with for feature extraction!

### Why Does This Matter?

Understanding sample rate is crucial because:
1. It tells us **how much data** we're working with
2. It determines **time-to-sample conversion** (e.g., "0.5 seconds = 8,000 audio samples")
3. It affects **window sizes** for machine learning (explained in the next section)

Let's verify our sensor sample rates:

In [ ]:
print(f"Audio sample rate: {FS_AUDIO} Hz")
print(f"IMU sample rate: {FS_IMU} Hz")

## Understanding Windowing - Fixed-Length Segments

### Why Do We Need Windows?

Machine learning models require **fixed-size inputs**. But our recordings vary in length:
- Some recordings are 10 seconds long
- Others are 30 seconds or longer
- A single cough might last 0.3 seconds, another might last 0.6 seconds

**The solution**: Extract fixed-length **windows** (segments) from the recordings.

### What is a Window?

A **window** is a fixed-length segment of the signal, like cutting a movie into clips of exactly 10 seconds each.

From the original paper:
- **Window length**: 0.4 seconds
- **Why 0.4 seconds?** This is long enough to capture a complete cough (typically 0.2-0.5s), but short enough to be efficient for ML

### Window Length ↔ Sample Count Relationship

The number of samples in a window depends on the **sampling frequency**:

**Formula**: `number_of_samples = window_length_seconds × sampling_frequency`

**For audio (16,000 Hz):**
- 0.4 seconds × 16,000 samples/second = **6,400 samples**

**For IMU (100 Hz):**
- 0.4 seconds × 100 samples/second = **40 samples**

**Analogy**: If you record video at 30 frames per second, a 2-second clip contains 2 × 30 = 60 frames. Same concept!

In [ ]:
def extract_sample_window(
        sample: np.ndarray,
        frequency: int, 
        window_duration: float,
        start_point_index = None,
        start_point_time = None,
    ):
    """
    Extract features from the given NumPy array, the window duration, and optionally, the starting point.
    The starting point can either be an index of the sample, or a the time that sample should start,
    which in this case will calculate the sample index automatically.

    ## Example:
    - Given the sample rate: 16000 Hz
    - If window starts at 0.5s → 0.5 * 16000 = 8000th sample
    - The window length to extract is 0.4s → 0.4 × 16000 = 6400 samples
    - Thus, the sample region to extract is feature[8000: 8000 + 6400]
    """
    window_length = int(window_duration * frequency)
    if start_point_index is None:
        if start_point_time is not None:
            # Calculate the sample start point from the given time
            start_point_index = int(start_point_time * frequency)
        else:
            raise ValueError("You must provide either start_point_index or start_point_time.")
    else:
        # Clamp to [0, sample_length - window_length]
        start_point_index = min(max(0, start_point_index), len(sample) - window_length)
    
    # Slice the sample using NumPy slicing
    return sample[start_point_index: start_point_index + window_length]

### Two Types of Windows We'll Extract

In this paper, we need to extract both cough and non-cough audios. Thus, we will have two types of windows.

1. **Cough windows**
   - We know the exact start/end time from `ground_truth.json`, so we can use it as a reference.
   - We will extract their windows by centering around annotated cough events
   - The window includes the cough, possibly shifted randomly for data augmentation

2. **Non-cough windows**:
   - There's no ground truth. We didn't cough!
   - We will randomly sampled segments from non-cough sounds (laugh, throat-clearing, breathing recordings)
   - Used as negative examples (label = 0)

In [ ]:
def extract_subject_window(audio, imu, start_point_time=None, start_point_index=None):
    """Extract audio and IMU window from the given window start point"""
    audio_window = extract_sample_window(audio, FS_AUDIO, WINDOW_LEN, start_point_index, start_point_time)

    # For IMU features, we need to stack all 6 IMU channels into a 2D array
    # Channels: [Accel_x, Accel_y, Accel_z, Gyro_Y, Gyro_P, Gyro_R]
    imu_window = np.column_stack([
        extract_sample_window(feature, FS_IMU, WINDOW_LEN, start_point_index, start_point_time)
        for feature in [imu.x, imu.y, imu.z, imu.Y, imu.P, imu.R]
    ])
    return audio_window, imu_window

---

## 🎓 Understanding Features Through Examples

We'll take **two real examples** from our dataset and walk through the feature extraction process step-by-step:

- **Example A**: A real **cough** from the dataset
- **Example B**: A **throat-clearing** sound (non-cough)

By comparing these side-by-side, you'll understand:
1. What each of the 105 features captures
2. Why these features help distinguish coughs from other sounds
3. How audio and IMU signals complement each other

This hands-on walkthrough will make the abstract concept of "features" concrete and intuitive!

### Step 0: Load dataset

First, start by selecting one subject from the dataset as an example, and perform window extraction on that subject.

In [ ]:
# Locate dataset folder
kaggle_dataset_dir = '/kaggle/input/edge-ai-cough-count'
base_dir = kaggle_dataset_dir if os.path.exists(kaggle_dataset_dir) else ".."
data_folder = base_dir + '/public_dataset/'

# Check if exists, otherwise try alternative path
if not os.path.exists(data_folder):
    raise FileNotFoundError(
        "Cannot find public_dataset/. Please download from: "
        "https://zenodo.org/record/7562332"
    )

# Get list of subject IDs
subject_ids = [d for d in os.listdir(data_folder) 
               if os.path.isdir(os.path.join(data_folder, d))]
subject_ids = sorted(subject_ids)

print(f"✓ Found {len(subject_ids)} subjects: {subject_ids}")

In [ ]:
# ===================================================================
# Load Example A: Cough Window
# ===================================================================
# Using subject "14287", trial 1, sitting, no noise, cough sound
example_subj = "14287"

# Load full audio signals (both microphones: air-facing and body-facing)
cough_audio_air, cough_audio_skin = load_audio(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.COUGH
)

# Load full IMU signals (all 6 channels: accel x,y,z + gyro Y,P,R)
cough_imu = load_imu(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.COUGH
)

# Load ground truth annotations (start/end times of each cough in seconds)
cough_starts, cough_ends = load_annotation(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.COUGH
)

cough_audio_window, cough_imu_window = extract_subject_window(
    audio=cough_audio_air,
    imu=cough_imu,
    # For window start point, we have ground truth on when the cough happens
    # Let's select the time when we first cough
    start_point_time=cough_starts[0]
)

# ===================================================================
# Load Example B: Throat-Clearing Window (Non-Cough)
# ===================================================================
# Load full throat-clearing signals
throat_audio_air, throat_audio_skin = load_audio(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.THROAT
)
throat_imu = load_imu(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.THROAT
)

throat_audio_window, throat_imu_window = extract_subject_window(
    audio=throat_audio_air,
    imu=throat_imu,
    # We don't have a starting point in mind, so pick randomly from the sample length
    start_point_index=np.random.randint(0, len(throat_audio_air))
)

print("✓ Loaded example windows:")
print(f"  Cough: {len(cough_audio_window)} audio samples, {len(cough_imu_window)} IMU samples")
print(f"  Throat-clearing: {len(throat_audio_window)} audio samples, {len(throat_imu_window)} IMU samples")
print(f"\n  Expected: 6400 audio samples (0.4s × 16kHz), 40 IMU samples (0.4s × 100Hz)")

### Step 1: Select and Visualize Example Windows

Let's look at the raw signals. Notice the differences in waveform patterns between cough and throat-clearing.

In [ ]:
# Visualize raw signals side-by-side
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Cough audio
time_audio = np.arange(len(cough_audio_window)) / FS_AUDIO
axes[0, 0].plot(time_audio, cough_audio_window, color='steelblue', linewidth=0.8)
axes[0, 0].set_title('Example A: Cough - Audio Signal', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Time (seconds)', fontsize=11)
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].grid(alpha=0.3)
# Add annotation for sharp burst
peak_idx = np.argmax(np.abs(cough_audio_window))
peak_val = cough_audio_window[peak_idx]
y_range = axes[0, 0].get_ylim()[1] - axes[0, 0].get_ylim()[0]
text_offset = y_range * 0.20 if peak_val > 0 else -y_range * 0.20
axes[0, 0].annotate('Sharp burst →', 
                    xy=(peak_idx/FS_AUDIO, peak_val),
                    xytext=(peak_idx/FS_AUDIO - 0.08, peak_val + text_offset),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2),
                    fontsize=10, color='red', fontweight='bold')

# Cough IMU (accelerometer Z)
time_imu = np.arange(len(cough_imu_window)) / FS_IMU
axes[0, 1].plot(time_imu, -cough_imu_window[:, 2], color='darkgreen', linewidth=1.5)
axes[0, 1].set_title('Example A: Cough - IMU Accel Z', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Time (seconds)', fontsize=11)
axes[0, 1].set_ylabel('Acceleration', fontsize=11)
axes[0, 1].grid(alpha=0.3)
# Add annotation for IMU spike
imu_peak_idx = np.argmax(-cough_imu_window[:, 2])
imu_peak_val = -cough_imu_window[imu_peak_idx, 2]
y_range_imu = axes[0, 1].get_ylim()[1] - axes[0, 1].get_ylim()[0]
text_offset_imu = y_range_imu * 0.20 if imu_peak_val > 0 else -y_range_imu * 0.20
axes[0, 1].annotate('← IMU spike', 
                    xy=(imu_peak_idx/FS_IMU, imu_peak_val),
                    xytext=(imu_peak_idx/FS_IMU + 0.05, imu_peak_val + text_offset_imu),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2),
                    fontsize=10, color='red', fontweight='bold')

# Throat-clearing audio
axes[1, 0].plot(time_audio, throat_audio_window, color='orange', linewidth=0.8)
axes[1, 0].set_title('Example B: Throat-Clearing - Audio Signal', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Time (seconds)', fontsize=11)
axes[1, 0].set_ylabel('Amplitude', fontsize=11)
axes[1, 0].grid(alpha=0.3)

# Throat-clearing IMU
axes[1, 1].plot(time_imu, -throat_imu_window[:, 2], color='purple', linewidth=1.5)
axes[1, 1].set_title('Example B: Throat-Clearing - IMU Accel Z', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Time (seconds)', fontsize=11)
axes[1, 1].set_ylabel('Acceleration', fontsize=11)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

#### 📊 Observations:
- Cough audio: Sharp, sudden burst with high amplitude
- Cough IMU: Clear spike corresponding to chest/body movement
- Throat-clearing: More gradual, sustained sound pattern
- Features will capture these differences quantitatively!

### Step 2: Understanding Audio - The Spectrogram

Audio signals have both **time** and **frequency** information. A **spectrogram** shows how frequencies change over time - like a musical score that displays which notes (frequencies) are played when.

**Think of it like this:**
- **Low frequencies** (bottom of spectrogram): Deep, bass sounds
- **High frequencies** (top of spectrogram): Sharp, treble sounds like "s" or "sh"
- **Color intensity**: How loud that frequency is at that moment

Let's visualize the spectrograms for our cough vs throat-clearing examples:

In [ ]:
# Create spectrograms
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cough spectrogram
D_cough = librosa.amplitude_to_db(np.abs(librosa.stft(cough_audio_window)), ref=np.max)
img1 = librosa.display.specshow(D_cough, sr=FS_AUDIO, x_axis='time', y_axis='hz', 
                                 ax=axes[0], cmap='viridis')
axes[0].set_title('Example A: Cough - Spectrogram', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Time (seconds)', fontsize=11)
axes[0].set_ylabel('Frequency (Hz)', fontsize=11)
axes[0].set_ylim(0, 8000)
fig.colorbar(img1, ax=axes[0], format='%+2.0f dB')

# Throat-clearing spectrogram
D_throat = librosa.amplitude_to_db(np.abs(librosa.stft(throat_audio_window)), ref=np.max)
img2 = librosa.display.specshow(D_throat, sr=FS_AUDIO, x_axis='time', y_axis='hz',
                                 ax=axes[1], cmap='viridis')
axes[1].set_title('Example B: Throat-Clearing - Spectrogram', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Time (seconds)', fontsize=11)
axes[1].set_ylabel('Frequency (Hz)', fontsize=11)
axes[1].set_ylim(0, 8000)
fig.colorbar(img2, ax=axes[1], format='%+2.0f dB')

plt.tight_layout()
plt.show()

#### 📊 What we observe:
- Cough: Sharp vertical bands (sudden burst) with strong high-frequency content (bright yellow at top)
- Throat-clearing: More horizontal spread (sustained) with energy concentrated in lower frequencies
- Coughs are like cymbal crashes - lots of high frequencies
- Throat-clearing is more like clearing vocal cords - lower, more gradual

### Step 3: MFCC Features - Capturing Audio "Texture"

**MFCC** stands for **Mel-Frequency Cepstral Coefficients** - a fancy name for features that capture the "shape" or "texture" of sound.

**Beginner-friendly explanation:**
- MFCCs are inspired by how human ears perceive sound
- The "Mel" scale mimics how we hear: we're better at distinguishing low frequencies than high ones
- We extract **13 MFCC values** at each time point, then compute statistics (mean, std, min, max) across the window
- This gives us **13 × 4 = 52 MFCC features** per audio window

**Why MFCCs matter:**
- MFCC coefficient 1-3 capture the overall spectral shape (coughs are sharper and more percussive)
- Higher coefficients capture fine details of the sound texture
- They're widely used in speech recognition and audio classification

In [ ]:
# Compute MFCCs for both examples
mfcc_cough = librosa.feature.mfcc(y=cough_audio_window, sr=FS_AUDIO, n_mfcc=13)
mfcc_throat = librosa.feature.mfcc(y=throat_audio_window, sr=FS_AUDIO, n_mfcc=13)

# Compute statistics for comparison
mfcc_cough_mean = np.mean(mfcc_cough, axis=1)
mfcc_throat_mean = np.mean(mfcc_throat, axis=1)
mfcc_cough_std = np.std(mfcc_cough, axis=1)
mfcc_throat_std = np.std(mfcc_throat, axis=1)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# MFCC heatmap for cough
img1 = axes[0, 0].imshow(mfcc_cough, aspect='auto', origin='lower', cmap='coolwarm')
axes[0, 0].set_title('Example A: Cough - MFCC Heatmap', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Time Frame', fontsize=11)
axes[0, 0].set_ylabel('MFCC Coefficient', fontsize=11)
axes[0, 0].set_yticks(range(13))
fig.colorbar(img1, ax=axes[0, 0])

# MFCC heatmap for throat-clearing
img2 = axes[0, 1].imshow(mfcc_throat, aspect='auto', origin='lower', cmap='coolwarm')
axes[0, 1].set_title('Example B: Throat-Clearing - MFCC Heatmap', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Time Frame', fontsize=11)
axes[0, 1].set_ylabel('MFCC Coefficient', fontsize=11)
axes[0, 1].set_yticks(range(13))
fig.colorbar(img2, ax=axes[0, 1])

# Bar chart of MFCC means
x = np.arange(13)
width = 0.35
axes[1, 0].bar(x - width/2, mfcc_cough_mean, width, label='Cough', color='steelblue', alpha=0.8)
axes[1, 0].bar(x + width/2, mfcc_throat_mean, width, label='Throat-clearing', color='orange', alpha=0.8)
axes[1, 0].set_title('MFCC Mean Comparison', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('MFCC Coefficient', fontsize=11)
axes[1, 0].set_ylabel('Mean Value', fontsize=11)
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3, axis='y')

# Bar chart of MFCC standard deviations
axes[1, 1].bar(x - width/2, mfcc_cough_std, width, label='Cough', color='steelblue', alpha=0.8)
axes[1, 1].bar(x + width/2, mfcc_throat_std, width, label='Throat-clearing', color='orange', alpha=0.8)
axes[1, 1].set_title('MFCC Standard Deviation Comparison', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('MFCC Coefficient', fontsize=11)
axes[1, 1].set_ylabel('Std Value', fontsize=11)
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

#### 📊 Key insights:
- Notice how coefficients 0-3 differ significantly between cough and throat-clearing
- These capture the overall spectral shape - coughs are sharper and more percussive
- The standard deviation (variability over time) also differs
- Together, these 52 features (13 coefficients × 4 statistics) provide a powerful signature!

### Step 4: Spectral Features - Frequency Characteristics

Spectral features describe **WHERE the energy is concentrated** in the frequency spectrum. Think of them as describing the "color" of sound.

**Beginner-friendly definitions:**
- **Spectral Centroid**: The "center of mass" of frequencies - like finding the average frequency. A high centroid means more high-frequency content (like "s" sounds).
- **Spectral Rolloff**: The frequency below which 85% of the energy is concentrated. Higher rolloff = more high-frequency energy.
- **Spectral Bandwidth**: The "spread" of frequencies - how wide the frequency range is.
- **Spectral Flatness**: How "noisy" vs "tonal" the sound is (white noise = 1, pure tone = 0).

Let's visualize these for our examples:

In [ ]:
# Compute spectral features

# Compute PSD (Power Spectral Density)
freqs_cough, psd_cough = signal.welch(cough_audio_window, fs=FS_AUDIO, nperseg=512)
freqs_throat, psd_throat = signal.welch(throat_audio_window, fs=FS_AUDIO, nperseg=512)

# Compute spectral centroid
centroid_cough = np.mean(librosa.feature.spectral_centroid(y=cough_audio_window, sr=FS_AUDIO))
centroid_throat = np.mean(librosa.feature.spectral_centroid(y=throat_audio_window, sr=FS_AUDIO))

# Compute spectral rolloff
rolloff_cough = np.mean(librosa.feature.spectral_rolloff(y=cough_audio_window, sr=FS_AUDIO))
rolloff_throat = np.mean(librosa.feature.spectral_rolloff(y=throat_audio_window, sr=FS_AUDIO))

# Compute spectral bandwidth
bandwidth_cough = np.mean(librosa.feature.spectral_bandwidth(y=cough_audio_window, sr=FS_AUDIO))
bandwidth_throat = np.mean(librosa.feature.spectral_bandwidth(y=throat_audio_window, sr=FS_AUDIO))

# Visualize PSD with spectral features
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cough PSD
axes[0].plot(freqs_cough, psd_cough, color='steelblue', linewidth=2, label='PSD')
axes[0].axvline(centroid_cough, color='red', linestyle='--', linewidth=2, label=f'Centroid: {centroid_cough:.0f} Hz')
axes[0].axvline(rolloff_cough, color='blue', linestyle='--', linewidth=2, label=f'Rolloff: {rolloff_cough:.0f} Hz')
axes[0].axvspan(centroid_cough - bandwidth_cough/2, centroid_cough + bandwidth_cough/2, 
                alpha=0.2, color='green', label=f'Bandwidth: {bandwidth_cough:.0f} Hz')
axes[0].set_title('Example A: Cough - Spectral Features', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Frequency (Hz)', fontsize=11)
axes[0].set_ylabel('Power Spectral Density', fontsize=11)
axes[0].set_xlim(0, 8000)
axes[0].legend(fontsize=9)
axes[0].grid(alpha=0.3)

# Throat-clearing PSD
axes[1].plot(freqs_throat, psd_throat, color='orange', linewidth=2, label='PSD')
axes[1].axvline(centroid_throat, color='red', linestyle='--', linewidth=2, label=f'Centroid: {centroid_throat:.0f} Hz')
axes[1].axvline(rolloff_throat, color='blue', linestyle='--', linewidth=2, label=f'Rolloff: {rolloff_throat:.0f} Hz')
axes[1].axvspan(centroid_throat - bandwidth_throat/2, centroid_throat + bandwidth_throat/2,
                alpha=0.2, color='green', label=f'Bandwidth: {bandwidth_throat:.0f} Hz')
axes[1].set_title('Example B: Throat-Clearing - Spectral Features', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Frequency (Hz)', fontsize=11)
axes[1].set_ylabel('Power Spectral Density', fontsize=11)
axes[1].set_xlim(0, 8000)
axes[1].legend(fontsize=9)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Create comparison table
print("\n📊 Spectral Feature Comparison:")
print(f"{'Feature':<20} {'Cough':>15} {'Throat-Clearing':>20}")
print("-" * 60)
print(f"{'Spectral Centroid':<20} {centroid_cough:>12.1f} Hz {centroid_throat:>17.1f} Hz")
print(f"{'Spectral Rolloff':<20} {rolloff_cough:>12.1f} Hz {rolloff_throat:>17.1f} Hz")
print(f"{'Spectral Bandwidth':<20} {bandwidth_cough:>12.1f} Hz {bandwidth_throat:>17.1f} Hz")

#### 💡 Insight:
- Coughs typically have HIGHER spectral centroid and rolloff (more high-frequency 'hiss')
- Throat-clearing has more energy in LOWER frequencies (vocal cord vibrations)
- These differences help the ML model distinguish coughs from other sounds!

### Step 5: Time-Domain Features - Temporal Characteristics

Time-domain features describe properties of the waveform **over time** (rather than in frequency).

**Beginner-friendly definitions:**
- **Zero-Crossing Rate (ZCR)**: How often the signal crosses zero (related to frequency). High ZCR = lots of oscillations.
- **RMS Energy**: Root Mean Square energy - a measure of "loudness" or overall power.
- **Crest Factor**: Ratio of peak amplitude to RMS. Measures how "peaky" or "spiky" the signal is.
  - High crest factor = sharp transients (like a cough)
  - Low crest factor = smooth, sustained sound (like a vowel)

---

**⚠️ Important Note on DC Offset and Preprocessing:**

According to the research paper methodology:
- **IMU signals**: DC offset (mean) is explicitly removed from each 0.4s window before feature extraction
- **Audio signals**: No DC offset removal is mentioned - features are computed on raw audio

**For the visualization cell below ONLY**, we remove DC offset from audio to improve visual clarity when comparing peak/RMS levels. This makes it easier to see the AC component (the actual oscillations we care about).

**The actual feature extraction functions compute features on raw audio signals** (with DC offset intact), staying faithful to the paper. The visualizations below are for educational purposes only.

Let's visualize these:

In [ ]:
# Compute time-domain features
# Zero-crossing rate
zcr_cough = librosa.feature.zero_crossing_rate(cough_audio_window)[0].mean()
zcr_throat = librosa.feature.zero_crossing_rate(throat_audio_window)[0].mean()

# RMS energy
rms_cough = np.sqrt(np.mean(cough_audio_window**2))
rms_throat = np.sqrt(np.mean(throat_audio_window**2))

# Crest factor
crest_cough = np.max(np.abs(cough_audio_window)) / (rms_cough + 1e-10)
crest_throat = np.max(np.abs(throat_audio_window)) / (rms_throat + 1e-10)

# ===================================================================
# For VISUALIZATION ONLY: Remove DC offset for clarity
# ===================================================================
cough_centered = cough_audio_window - np.mean(cough_audio_window)
throat_centered = throat_audio_window - np.mean(throat_audio_window)

# Compute visualization-only metrics on DC-centered signals
zcr_cough_vis = np.sum(np.diff(np.sign(cough_centered)) != 0) / len(cough_centered)
zcr_throat_vis = np.sum(np.diff(np.sign(throat_centered)) != 0) / len(throat_centered)

rms_cough_vis = np.sqrt(np.mean(cough_centered**2))
rms_throat_vis = np.sqrt(np.mean(throat_centered**2))

peak_cough_vis = np.max(np.abs(cough_centered))
peak_throat_vis = np.max(np.abs(throat_centered))

crest_cough_vis = peak_cough_vis / (rms_cough_vis + 1e-10)
crest_throat_vis = peak_throat_vis / (rms_throat_vis + 1e-10)

# ===================================================================
# Visualize
# ===================================================================
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

# Row 1: Waveforms with zero crossings (DC-centered)
time_audio = np.arange(len(cough_audio_window)) / FS_AUDIO

axes[0, 0].plot(time_audio, cough_centered, color='steelblue', linewidth=0.8, label='Waveform (DC removed)')
axes[0, 0].axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
zero_crossings_cough = np.where(np.diff(np.sign(cough_centered)))[0]
axes[0, 0].scatter(zero_crossings_cough[:100]/FS_AUDIO, 
                   np.zeros(min(100, len(zero_crossings_cough))), 
                   color='red', s=10, alpha=0.5, label='Zero crossings (first 100)')
axes[0, 0].set_title(f'Example A: Cough - Zero Crossings (ZCR={zcr_cough_vis:.4f})', 
                     fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Time (seconds)', fontsize=10)
axes[0, 0].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(alpha=0.3)

axes[0, 1].plot(time_audio, throat_centered, color='orange', linewidth=0.8, label='Waveform (DC removed)')
axes[0, 1].axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
zero_crossings_throat = np.where(np.diff(np.sign(throat_centered)))[0]
axes[0, 1].scatter(zero_crossings_throat[:100]/FS_AUDIO,
                   np.zeros(min(100, len(zero_crossings_throat))),
                   color='red', s=10, alpha=0.5, label='Zero crossings (first 100)')
axes[0, 1].set_title(f'Example B: Throat-Clearing - Zero Crossings (ZCR={zcr_throat_vis:.4f})',
                     fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Time (seconds)', fontsize=10)
axes[0, 1].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[0, 1].legend(fontsize=9)
axes[0, 1].grid(alpha=0.3)

# Row 2: RMS Energy comparison (visualization values)
axes[1, 0].bar(['Cough', 'Throat-Clearing'], [rms_cough_vis, rms_throat_vis], 
               color=['steelblue', 'orange'], alpha=0.7)
axes[1, 0].set_title('RMS Energy Comparison (DC removed)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('RMS Energy', fontsize=10)
axes[1, 0].grid(alpha=0.3, axis='y')
for i, (label, val) in enumerate([('Cough', rms_cough_vis), ('Throat', rms_throat_vis)]):
    axes[1, 0].text(i, val, f'{val:.4f}', ha='center', va='bottom', fontsize=10)

# Row 2: Crest Factor comparison (visualization values)
axes[1, 1].bar(['Cough', 'Throat-Clearing'], [crest_cough_vis, crest_throat_vis],
               color=['steelblue', 'orange'], alpha=0.7)
axes[1, 1].set_title('Crest Factor Comparison (DC removed)', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Crest Factor (Peak/RMS)', fontsize=10)
axes[1, 1].grid(alpha=0.3, axis='y')
for i, (label, val) in enumerate([('Cough', crest_cough_vis), ('Throat', crest_throat_vis)]):
    axes[1, 1].text(i, val, f'{val:.2f}', ha='center', va='bottom', fontsize=10)

# Row 3: Waveforms with peak and RMS lines (DC-centered)
axes[2, 0].plot(time_audio, cough_centered, color='steelblue', linewidth=0.8, label='Waveform (DC removed)')
axes[2, 0].axhline(rms_cough_vis, color='green', linestyle='--', linewidth=2, label=f'RMS: {rms_cough_vis:.4f}')
axes[2, 0].axhline(-rms_cough_vis, color='green', linestyle='--', linewidth=2)
axes[2, 0].axhline(peak_cough_vis, color='red', linestyle='--', linewidth=2, label=f'Peak: {peak_cough_vis:.4f}')
axes[2, 0].axhline(-peak_cough_vis, color='red', linestyle='--', linewidth=2)
axes[2, 0].set_title(f'Example A: Cough - Peak vs RMS (Crest={crest_cough_vis:.2f})',
                     fontsize=12, fontweight='bold')
axes[2, 0].set_xlabel('Time (seconds)', fontsize=10)
axes[2, 0].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[2, 0].legend(fontsize=9)
axes[2, 0].grid(alpha=0.3)

axes[2, 1].plot(time_audio, throat_centered, color='orange', linewidth=0.8, label='Waveform (DC removed)')
axes[2, 1].axhline(rms_throat_vis, color='green', linestyle='--', linewidth=2, label=f'RMS: {rms_throat_vis:.4f}')
axes[2, 1].axhline(-rms_throat_vis, color='green', linestyle='--', linewidth=2)
axes[2, 1].axhline(peak_throat_vis, color='red', linestyle='--', linewidth=2, label=f'Peak: {peak_throat_vis:.4f}')
axes[2, 1].axhline(-peak_throat_vis, color='red', linestyle='--', linewidth=2)
axes[2, 1].set_title(f'Example B: Throat-Clearing - Peak vs RMS (Crest={crest_throat_vis:.2f})',
                     fontsize=12, fontweight='bold')
axes[2, 1].set_xlabel('Time (seconds)', fontsize=10)
axes[2, 1].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[2, 1].legend(fontsize=9)
axes[2, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# ===================================================================
# Print comparison table using ACTUAL features (raw signal)
# ===================================================================
print("\n📊 Time-Domain Feature Comparison (ACTUAL features used by ML model):")
print(f"{'Feature':<20} {'Cough':>15} {'Throat-Clearing':>20}")
print("-" * 60)
print(f"{'Zero-Crossing Rate':<20} {zcr_cough:>15.4f} {zcr_throat:>20.4f}")
print(f"{'RMS Energy':<20} {rms_cough:>15.4f} {rms_throat:>20.4f}")
print(f"{'Crest Factor':<20} {crest_cough:>15.2f} {crest_throat:>20.2f}")
print(f"{'DC Offset (mean)':<20} {np.mean(cough_audio_window):>15.4f} {np.mean(throat_audio_window):>20.4f}")

print("\n📊 Visualization-Only Metrics (DC-centered for clarity):")
print(f"{'Feature':<20} {'Cough':>15} {'Throat-Clearing':>20}")
print("-" * 60)
print(f"{'Zero-Crossing Rate':<20} {zcr_cough_vis:>15.4f} {zcr_throat_vis:>20.4f}")
print(f"{'RMS Energy':<20} {rms_cough_vis:>15.4f} {rms_throat_vis:>20.4f}")
print(f"{'Crest Factor':<20} {crest_cough_vis:>15.2f} {crest_throat_vis:>20.2f}")

#### 💡 Insight:
- Coughs have HIGH crest factor (sharp peaks) - characteristic of impulsive sounds
- Throat-clearing has LOWER crest factor - more gradual and sustained
- Higher ZCR in coughs reflects the noisy, broadband nature of the sound

#### ⚠️ Note:
- ACTUAL ML features are computed on raw audio (with DC offset)
- Visualizations use DC-centered audio for clarity only
- This follows the paper methodology: no DC removal for audio, but DC removal for IMU

### Step 6: IMU Features - Motion Signals

Now let's look at the **Inertial Measurement Unit (IMU)** - the motion sensors.

**Beginner-friendly explanation:**
- **IMU**: A sensor that measures motion using two components:
  - **Accelerometer**: Measures linear acceleration (like feeling pushed in a car). Captures chest/body movement during cough.
  - **Gyroscope**: Measures rotation (like turning your head). Captures head/neck movement during cough.

- **Each sensor has 3 axes**: X, Y, Z (measuring motion in 3D space)
- **L2 norm**: The total magnitude of motion, calculated as √(x² + y² + z²)
  - Think of it like calculating distance: if you walk 3 meters north and 4 meters east, you're √(3² + 4²) = 5 meters from your starting point

**Total**: 3 accel + 1 accel_L2 + 3 gyro + 1 gyro_L2 = **8 signals** × 5 features each = **40 IMU features**

Let's visualize all 8 signals:

In [ ]:
# Compute L2 norms
# First, center the signals (subtract mean) as per the paper
cough_imu_centered = cough_imu_window - np.mean(cough_imu_window, axis=0, keepdims=True)
throat_imu_centered = throat_imu_window - np.mean(throat_imu_window, axis=0, keepdims=True)

# Compute L2 norms
cough_accel_l2 = np.linalg.norm(cough_imu_centered[:, 0:3], axis=1)
cough_gyro_l2 = np.linalg.norm(cough_imu_centered[:, 3:6], axis=1)
throat_accel_l2 = np.linalg.norm(throat_imu_centered[:, 0:3], axis=1)
throat_gyro_l2 = np.linalg.norm(throat_imu_centered[:, 3:6], axis=1)

# Build per-feature signal arrays (same order for both classes)
time_imu = np.arange(40) / FS_IMU
signal_names = ['Accel X', 'Accel Y', 'Accel Z', 'Accel L2', 'Gyro Y', 'Gyro P', 'Gyro R', 'Gyro L2']
colors = ['steelblue', 'darkgreen', 'purple', 'red', 'orange', 'brown', 'pink', 'darkred']

cough_signals = [
    cough_imu_centered[:, 0],  # Accel X
    cough_imu_centered[:, 1],  # Accel Y
    cough_imu_centered[:, 2],  # Accel Z
    cough_accel_l2,            # Accel L2
    cough_imu_centered[:, 3],  # Gyro Y
    cough_imu_centered[:, 4],  # Gyro P
    cough_imu_centered[:, 5],  # Gyro R
    cough_gyro_l2              # Gyro L2
]

throat_signals = [
    throat_imu_centered[:, 0],  # Accel X
    throat_imu_centered[:, 1],  # Accel Y
    throat_imu_centered[:, 2],  # Accel Z
    throat_accel_l2,            # Accel L2
    throat_imu_centered[:, 3],  # Gyro Y
    throat_imu_centered[:, 4],  # Gyro P
    throat_imu_centered[:, 5],  # Gyro R
    throat_gyro_l2              # Gyro L2
]

# --- One figure: 8 rows (features) × 2 cols (Cough vs Throat) ---
fig, axes = plt.subplots(
    nrows=8, ncols=2,
    figsize=(16, 18),
    sharex=True
)

for i, (name, color) in enumerate(zip(signal_names, colors)):
    # Left column: Cough
    axL = axes[i, 0]
    axL.plot(time_imu, cough_signals[i], color=color, linewidth=2)
    axL.set_title(f'Cough - {name}', fontsize=11, fontweight='bold')
    axL.set_ylabel('Magnitude', fontsize=9)
    axL.grid(alpha=0.3)
    axL.axhline(0, color='black', linestyle='--', linewidth=0.5, alpha=0.5)

    # Right column: Throat-clearing
    axR = axes[i, 1]
    axR.plot(time_imu, throat_signals[i], color=color, linewidth=2)
    axR.set_title(f'Throat-Clearing - {name}', fontsize=11, fontweight='bold')
    axR.grid(alpha=0.3)
    axR.axhline(0, color='black', linestyle='--', linewidth=0.5, alpha=0.5)

# Only bottom row gets x-labels to reduce clutter
axes[-1, 0].set_xlabel('Time (seconds)', fontsize=9)
axes[-1, 1].set_xlabel('Time (seconds)', fontsize=9)

plt.suptitle('IMU Features - Motion Signals (Cough vs Throat-Clearing)', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

#### 📊 Observations:
- Cough creates distinctive motion patterns:
    - Sudden chest movement (accelerometer)
    - Head/neck movement (gyroscope)
- L2 norms capture the total magnitude regardless of direction
- These motion signatures complement audio features for robust detection

### Step 7: IMU Statistical Features

For each of the 8 IMU signals, we compute **5 statistical features**:

**Beginner-friendly definitions:**
- **Line Length**: Sum of absolute differences between consecutive samples. Measures how "wiggly" or active the signal is.
- **Zero-Crossing Rate**: How often the signal changes sign (crosses zero). Similar to audio ZCR.
- **Kurtosis**: A measure of "peakiness". High kurtosis = sharp spikes; low kurtosis = smooth signal.
- **Crest Factor**: Peak-to-RMS ratio (same as audio). Measures impulsiveness.
- **RMS**: Root Mean Square - overall magnitude of the signal.

Let's extract all 40 features and compare cough vs throat-clearing:

In [ ]:
# Extract full 40 IMU features using our feature extraction function
imu_features_cough = extract_imu_features(cough_imu_window)
imu_features_throat = extract_imu_features(throat_imu_window)

# Create feature names for visualization
feature_types = ['Line Length', 'ZCR', 'Kurtosis', 'Crest', 'RMS']
signal_names_short = ['Acc_X', 'Acc_Y', 'Acc_Z', 'Acc_L2', 'Gyro_Y', 'Gyro_P', 'Gyro_R', 'Gyro_L2']
feature_names = []
for sig in signal_names_short:
    for feat in feature_types:
        feature_names.append(f'{sig}_{feat}')

# Visualize all 40 features as a bar chart
fig, ax = plt.subplots(figsize=(16, 8))
x = np.arange(40)
width = 0.35

# Use raw feature values (no normalization for 2-sample comparison)
ax.bar(x - width/2, imu_features_cough, width, label='Cough', color='steelblue', alpha=0.7)
ax.bar(x + width/2, imu_features_throat, width, label='Throat-Clearing', color='orange', alpha=0.7)

ax.set_title('All 40 IMU Features Comparison (Raw Values)', fontsize=14, fontweight='bold')
ax.set_xlabel('Feature Index', fontsize=12)
ax.set_ylabel('Feature Value', fontsize=12)
ax.set_xticks(x[::5])  # Show every 5th tick
ax.set_xticklabels(range(0, 40, 5))
ax.legend(fontsize=11)
ax.grid(alpha=0.3, axis='y')
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)

# Add vertical lines to separate signal groups
for i in range(1, 8):
    ax.axvline(i * 5 - 0.5, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

# Add signal labels
signal_positions = [2.5, 7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5]
for pos, sig_name in zip(signal_positions, signal_names_short):
    ax.text(pos, ax.get_ylim()[1] * 0.9, sig_name, ha='center', fontsize=8,
            rotation=0, fontweight='bold', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout()
plt.show()

# Find top 10 most discriminative features (largest absolute difference)
differences = np.abs(imu_features_cough - imu_features_throat)
top_indices = np.argsort(differences)[-10:][::-1]

print("\n📊 Top 10 Most Discriminative IMU Features:")
print(f"{'Rank':<6} {'Feature Name':<25} {'Cough':>12} {'Throat':>12} {'Diff':>12}")
print("-" * 75)
for rank, idx in enumerate(top_indices, 1):
    print(f"{rank:<6} {feature_names[idx]:<25} {imu_features_cough[idx]:>12.4f} {imu_features_throat[idx]:>12.4f} {differences[idx]:>12.4f}")

#### 💡 Key Insights:

- **Gyroscope Line Length dominates** - all top 5 features measure rotational motion intensity
- Coughs produce **3x more gyroscope activity** than throat-clearing (head/neck rotation)
- Line Length (signal 'wiggliness') is far more discriminative than statistical measures
- Gyro_R (Roll) shows the largest difference - captures head tilting during cough
- IMU captures biomechanics: coughs = sudden head/neck movement, throat-clearing = more stationary
- These 40 features complement audio features for robust multimodal detection!

### Step 8: Complete Feature Vector - Putting It All Together

Now let's extract the **complete 105-feature vector** (65 audio + 40 IMU) for both examples and visualize the final result.

This is what gets fed into the machine learning model!

In [ ]:
# Extract complete feature vectors
audio_features_cough = extract_audio_features(cough_audio_window, fs=FS_AUDIO)
audio_features_throat = extract_audio_features(throat_audio_window, fs=FS_AUDIO)

# Combine audio + IMU
complete_features_cough = np.concatenate([audio_features_cough, imu_features_cough])
complete_features_throat = np.concatenate([audio_features_throat, imu_features_throat])

print(f"✓ Complete feature vectors extracted:")
print(f"  Cough: {len(complete_features_cough)} features")
print(f"  Throat-clearing: {len(complete_features_throat)} features")
print(f"\n  Breakdown: 65 audio + 40 IMU = 105 total\n")

# Visualize as grouped bar chart
fig, axes = plt.subplots(3, 1, figsize=(16, 14))

# Use log scale for better visualization of features with different magnitudes
# Apply log(1 + abs(x)) * sign(x) transformation to preserve sign
def log_transform(x):
    return np.sign(x) * np.log1p(np.abs(x))

cough_log = log_transform(complete_features_cough)
throat_log = log_transform(complete_features_throat)

# Panel 1: All 105 features
x = np.arange(105)
width = 0.35
axes[0].bar(x - width/2, cough_log, width, label='Cough', color='steelblue', alpha=0.7)
axes[0].bar(x + width/2, throat_log, width, label='Throat-Clearing', color='orange', alpha=0.7)
axes[0].set_title('Complete 105-Feature Vector (Log-Scaled)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature Index', fontsize=12)
axes[0].set_ylabel('Log-Scaled Feature Value', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3, axis='y')
axes[0].axhline(0, color='black', linestyle='-', linewidth=0.5)
# Add divider between audio and IMU
axes[0].axvline(64.5, color='red', linestyle='--', linewidth=2, label='Audio | IMU')
axes[0].text(32, axes[0].get_ylim()[1] * 0.85, 'Audio Features (65)', 
             ha='center', fontsize=11, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
axes[0].text(82, axes[0].get_ylim()[1] * 0.85, 'IMU Features (40)',
             ha='center', fontsize=11, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

# Panel 2: Audio features only (first 65)
x_audio = np.arange(65)
axes[1].bar(x_audio - width/2, cough_log[:65], width, label='Cough', color='steelblue', alpha=0.7)
axes[1].bar(x_audio + width/2, throat_log[:65], width, label='Throat-Clearing', color='orange', alpha=0.7)
axes[1].set_title('Audio Features Only (65 features, Log-Scaled)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Feature Index', fontsize=11)
axes[1].set_ylabel('Log-Scaled Feature Value', fontsize=11)
axes[1].legend(fontsize=10)
axes[1].grid(alpha=0.3, axis='y')
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.5)
# Add labels for MFCC, Spectral, Time-domain regions
axes[1].text(26, axes[1].get_ylim()[1] * 0.85, 'MFCC (52)', ha='center', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
axes[1].text(57, axes[1].get_ylim()[1] * 0.85, 'Spectral (10)', ha='center', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.4))
axes[1].text(63, axes[1].get_ylim()[1] * 0.85, 'Time (3)', ha='center', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.4))

# Panel 3: IMU features only (last 40)
x_imu = np.arange(40)
axes[2].bar(x_imu - width/2, cough_log[65:], width, label='Cough', color='steelblue', alpha=0.7)
axes[2].bar(x_imu + width/2, throat_log[65:], width, label='Throat-Clearing', color='orange', alpha=0.7)
axes[2].set_title('IMU Features Only (40 features, Log-Scaled)', fontsize=13, fontweight='bold')
axes[2].set_xlabel('Feature Index', fontsize=11)
axes[2].set_ylabel('Log-Scaled Feature Value', fontsize=11)
axes[2].legend(fontsize=10)
axes[2].grid(alpha=0.3, axis='y')
axes[2].axhline(0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# Compute overall statistics using raw values
differences_all = np.abs(complete_features_cough - complete_features_throat)
top_15_indices = np.argsort(differences_all)[-15:][::-1]

# Map feature indices to types
feature_type_map = {
    (0, 52): "MFCC",
    52: "Spectral Centroid",
    53: "Spectral Rolloff", 
    54: "Spectral Bandwidth",
    55: "Spectral Flatness",
    56: "Spectral Contrast",
    57: "Total Power",
    58: "Dominant Frequency",
    59: "Spectral Spread",
    60: "Spectral Skewness",
    61: "Spectral Kurtosis",
    62: "Zero-Crossing Rate",
    63: "RMS Energy",
    64: "Crest Factor",
}

print("\n" + "="*90)
print("📊 TOP 15 MOST DISCRIMINATIVE FEATURES (out of 105)")
print("="*90)
print(f"{'Rank':<6} {'Type':<10} {'Feature Name':<30} {'Cough':>15} {'Throat':>15}")
print("-" * 90)
for rank, idx in enumerate(top_15_indices, 1):
    if idx < 65:
        feat_type = "Audio"
        if idx < 52:
            feat_name = f"MFCC #{idx}"
        else:
            feat_name = feature_type_map.get(idx, f"Audio #{idx}")
    else:
        feat_type = "IMU"
        imu_idx = idx - 65
        sig_idx = imu_idx // 5
        feat_idx = imu_idx % 5
        feat_name = f"{signal_names_short[sig_idx]}_{feature_types[feat_idx]}"
    
    print(f"{rank:<6} {feat_type:<10} {feat_name:<30} {complete_features_cough[idx]:>15.2f} "
          f"{complete_features_throat[idx]:>15.2f}")

#### 🎯 CRITICAL INSIGHTS FROM THIS EXAMPLE

1. **Audio features dominate discrimination** - all top 15 are audio, NO IMU in top 15!

2. **Spectral features >> MFCC features**:
 - Spectral Kurtosis (rank 1): Measures frequency distribution 'peakiness'
 - Spectral Skewness (rank 2): Measures frequency distribution asymmetry
 - Spectral Rolloff (rank 3): 6x higher for cough (2547 vs 414 Hz)
 - Spectral Centroid (rank 4): 3x higher for cough (988 vs 312 Hz)

3. **Why spectral features win in THIS example**:
 - This is a quiet environment (sitting, no background noise)
 - Coughs have sharp, broadband spectral bursts (high kurtosis/skewness)
 - Throat-clearing has concentrated low-frequency energy
 - The spectral difference is dramatic and easily separable

4. **Context matters - this is ONE example**:
 - In noisy environments (music, traffic), IMU features become MORE critical
 - Multimodal (audio + IMU) provides robustness across ALL conditions
 - This notebook shows feature extraction; actual model uses ALL 105 features

5. **Next step**: Train XGBoost on the full dataset → Model_Training_XGBoost.ipynb
 - You'll see how feature importance changes across 15 subjects and all conditions
 - The model learns when to rely on audio vs IMU vs both

---

## 📚 Summary and Next Steps

### What We Accomplished

In this notebook, we transformed raw biosignal data into machine learning-ready features:

**Raw Data → Features:**
- **Input**: 0.4-second windows (6,400 audio samples + 40 IMU samples per window)
- **Output**: 105 compact features per window
- **Compression**: From ~6,440 values → 105 values (61× reduction!)

### Feature Breakdown

| Category | Count | Purpose |
|----------|-------|---------|
| **MFCC** | 52 | Capture sound texture/timbre |
| **Spectral** | 10 | Describe frequency distribution |
| **Time-domain** | 3 | Measure loudness and sharpness |
| **IMU** | 40 | Capture body motion patterns |
| **TOTAL** | **105** | Complete multimodal signature |

### Key Insights from Our Examples

Through our step-by-step walkthrough, we learned:

1. **Coughs have distinct characteristics:**
   - Sharp, impulsive audio bursts (high crest factor)
   - Strong high-frequency content (high spectral centroid/rolloff)
   - Distinctive motion patterns (accelerometer + gyroscope spikes)

2. **Multimodal sensing is powerful:**
   - Audio alone: Good, but struggles in noisy environments
   - IMU alone: Captures motion, but misses acoustic details
   - Combined: Robust detection across all conditions

3. **Features enable machine learning:**
   - Each window becomes a point in 105-dimensional space
   - Coughs cluster in one region, non-coughs in another
   - ML models learn to find the optimal decision boundary
---

**You now understand the complete feature extraction pipeline!** 🎉

The journey from raw biosignals → features → trained model is a fundamental workflow in biomedical signal processing and machine learning. The skills you've learned here apply to many other domains: speech recognition, activity recognition, ECG analysis, and more.